<a href="https://colab.research.google.com/github/DogaSahin/Twitter_Sentiment_Analysis/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy
!pip install textblob
!pip install preprocessor
!pip install statistics

  Created wheel for preprocessor: filename=preprocessor-1.1.3-cp37-none-any.whl size=4477 sha256=0b56d4e19327c449a2b469bdb000f43f34666108b44760b9043f8e27df844fe1
  Stored in directory: /root/.cache/pip/wheels/98/c1/a2/21fbcfd80d76576bbf148991a66f00730f541f265c7600000f
Successfully built preprocessor
  Created wheel for statistics: filename=statistics-1.0.3.5-cp37-none-any.whl size=7454 sha256=818b7857cc7be76180a6faad7829d76c581dbf2dc0bc913236d480cb74311a06
  Stored in directory: /root/.cache/pip/wheels/75/55/90/73aa7662bfb4565b567618547a275f01372a678ca92ecd64f3
Successfully built statistics


In [ ]:
import tweepy as tw
import textblob
import preprocessor
import statistics
import numpy as np
import pandas as pd
from typing import List

In [ ]:
api_key = "zs1VrItXow1kXKZhNtnbn8OQD"
api_secret_key = "CW47HJK9SnYjJTNVsMMdCfyjK0x3N8hKFTAXCSaTbpmqBMMMMI"
access_token = "1206982141172764672-3hFfVWIWYzz8dEuJnGnszPdhGY5FEE"
access_token_secret = "X7IXHwBdXKAix3BdmlZ9mhWSxXfB9OPFQwGstT7dSIMaa"
Bearer_token: "AAAAAAAAAAAAAAAAAAAAAMYGNwEAAAAAOcxExg4tCZGNHJ9tiUlvK3FsM58%3DOPbPEUBUDSag04nkiHSf8JWAdts7qkC34GUgeFNKrRXDk2MDMn"

auth = tw.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [ ]:
def get_tweets(keyword: str)->List[str]:
  all_tweets = []
  for tweet in tw.Cursor(api.search, q = keyword, tweet_mode = 'extended',lang='en').items(10):
    all_tweets.append(tweet.full_text)

  return all_tweets

def cleaning_tweets(all_tweets:List[str])->List[str]:
  clean_tweets=[]
  for tweet in all_tweets:
    clean_tweets.append(p.clean(tweet))

  return clean_tweets

keywords = ["covid","coronavirus","covid19"]
get_tweets(keywords)


['Updated: @KFF brief on global funding across the six #COVID19 emergency supplemental bills. \n\nThe most recent bill, the American Rescue Plan Act, includes the most funding for the global response to date.\n\nhttps://t.co/tzVn2TjG8O',
 'RT @ROWPublicHealth: Keep practicing public health measures. COVID-19 spreads easily between loved ones and friends, those with whom we may…',
 "Better late than never! SMH\n\nCDC's new school distancing guidelines say 3 feet apart is often enough\n\nhttps://t.co/uhHSlW4eUe  #coronavirus #COVID19 #vaccines #schoolreopening",
 'Swift claims settlement is key to rebuilding Covid-hit reputation says legal alliance @gilawconnect\n\nhttps://t.co/IF7nlKppXC\n\n#insurance #pandemic #coronavirus #covid19 #claims #claimspayouts #businessinterruption #reputation',
 'RT @CPHO_Canada: Today’s #COVID19 in 🇨🇦 info and updates:\n➡️#GOC website on #coronavirus disease: \nhttps://t.co/8kWHvr0h7S\n➡️Canada COVID-1…',
 'The @WMO has warned that #weather &amp; #climate 